# Load point-geometries of European capitals into PostGIS

This is a simplified example. Coordinates are extracted form GeoJSON, even if there are other ways to load this more efficient into PostGIS.

In [1]:
import json
import os
import psycopg2
import requests

In [2]:
# Download json file with coordinates of European capitals
url = 'https://raw.githubusercontent.com/Ginden/capitals/master/europe.json'
r = requests.get(url)

with open('/download/europe.json', 'wb') as f:
    f.write(r.content)

In [3]:
host = os.environ['PGHOST']
database = os.environ['PGDATABASE']
user = os.environ['PGUSER']
password = os.environ['PGPASSWORD']

In [4]:
connection_string = f'host={host} dbname={database} user={user} password={password}'

In [5]:
# Helper function to execute simple SQL statements

def execute_simple_sql(sql, fetch='None'):
    with psycopg2.connect(connection_string) as conn:
        with conn.cursor() as curs:
            if fetch=='None':
                curs.execute(sql)
            elif fetch=='One':
                curs.execute(sql)
                result = curs.fetchone();
                return result

In [6]:
# Create PostGIS extenstion

sql = 'CREATE EXTENSION IF NOT EXISTS postgis'
execute_simple_sql(sql)

In [7]:
# Create Postgres table

sql='CREATE TABLE IF NOT EXISTS capitals(gid serial PRIMARY KEY, name TEXT, point geography(POINT,4326) );'
execute_simple_sql(sql)

In [8]:
# Loop over capitals in json file and insert point by point
# (Please do not use the following function in a loop over a larger dataset)

with open('/download/europe.json') as f:
    data = json.load(f)
    for a_capital in data:
        capital_name = a_capital['properties']['capital']
        lon = a_capital['geometry']['coordinates'][0]
        lat = a_capital['geometry']['coordinates'][1]
        sql = f"INSERT INTO capitals (name, point) VALUES ('{capital_name}', 'SRID=4326;POINT({lon} {lat})');"
        execute_simple_sql(sql)

Check the results in PgAdmin

![Screenshot of PgAdmin](./images/Capitals_in_PgAdmin.png)